# setting enviroment

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

In [2]:
spark

In [3]:

!pip install kaggle


from google.colab import files
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d mkechinov/ecommerce-purchase-history-from-electronics-store


!unzip ecommerce-purchase-history-from-electronics-store.zip -d ecommerce_dataset

!ls ecommerce_dataset


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store
License(s): copyright-authors
 81% 41.0M/50.5M [00:01<00:00, 50.4MB/s]
100% 50.5M/50.5M [00:01<00:00, 50.4MB/s]
Archive:  ecommerce-purchase-history-from-electronics-store.zip
  inflating: ecommerce_dataset/kz.csv  
kz.csv


In [4]:
# Step 1: Install kagglehub
!pip install kagglehub

# Step 2: Import kagglehub and download dataset
import kagglehub

path = kagglehub.dataset_download("mkechinov/ecommerce-purchase-history-from-electronics-store")
print("Path to dataset files:", path)


100%|██████████| 50.5M/50.5M [00:00<00:00, 164MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/mkechinov/ecommerce-purchase-history-from-electronics-store/versions/2
                event_time             order_id           product_id  \
0  2020-04-24 11:50:39 UTC  2294359932054536986  1515966223509089906   
1  2020-04-24 11:50:39 UTC  2294359932054536986  1515966223509089906   
2  2020-04-24 14:37:43 UTC  2294444024058086220  2273948319057183658   
3  2020-04-24 14:37:43 UTC  2294444024058086220  2273948319057183658   
4  2020-04-24 19:16:21 UTC  2294584263154074236  2273948316817424439   

    category_id                category_code    brand   price       user_id  
0  2.268105e+18           electronics.tablet  samsung  162.01  1.515916e+18  
1  2.268105e+18           electronics.tablet  samsung  162.01  1.515916e+18  
2  2.268105e+18  electronics.audio.headphone   huawei   77.52  1.515916e+18  
3  2.268105e+18  electronics.audio.headphone   huawei   77.52  1.515916e+18  
4  2.268105e+18                          NaN  karch

#Spark Part to read data and prepare

In [5]:
import sys
from pyspark.sql import SparkSession, functions, types
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import current_date, year
from pyspark.sql.functions import to_timestamp

#Data Schema

In [6]:

transactions_schema = types.StructType([
    types.StructField('event_time', types.TimestampType()),
    types.StructField('order_id', types.StringType()),
    types.StructField('product_id', types.StringType()),
    types.StructField('category_id', types.StringType()),
    types.StructField('category_code', types.StringType()),
    types.StructField('brand', types.StringType()),
    types.StructField('price', types.FloatType()),
    types.StructField('user_id', types.StringType())
])

# import data

In [7]:
transactions = spark.read.csv(path + "/kz.csv", schema=transactions_schema, header=True)

In [18]:
origin_num_rows = transactions.count()
print(f"Number of rows in transactions: {origin_num_rows}")

Number of rows in transactions: 2632846


In [8]:
transactions.show(10, truncate=False)

+-------------------+-------------------+-------------------+-------------------+---------------------------+-------+-------+-------------------+
|event_time         |order_id           |product_id         |category_id        |category_code              |brand  |price  |user_id            |
+-------------------+-------------------+-------------------+-------------------+---------------------------+-------+-------+-------------------+
|2020-04-24 11:50:39|2294359932054536986|1515966223509089906|2268105426648170900|electronics.tablet         |samsung|162.01 |1515915625441993984|
|2020-04-24 11:50:39|2294359932054536986|1515966223509089906|2268105426648170900|electronics.tablet         |samsung|162.01 |1515915625441993984|
|2020-04-24 14:37:43|2294444024058086220|2273948319057183658|2268105430162997728|electronics.audio.headphone|huawei |77.52  |1515915625447879434|
|2020-04-24 14:37:43|2294444024058086220|2273948319057183658|2268105430162997728|electronics.audio.headphone|huawei |77.52  

# Counting the number of product_id, category_id, category_code, brand


In [9]:

product_count = transactions.select("product_id").distinct().count()
category_id_count = transactions.select("category_id").distinct().count()
category_code_count = transactions.select("category_code").distinct().count()
brand_count = transactions.select("brand").distinct().count()


In [10]:

print(f"Unique product_id count: {product_count}")
print(f"Unique category_id count: {category_id_count}")
print(f"Unique category_code count: {category_code_count}")
print(f"Unique brand count: {brand_count}")


Unique product_id count: 25113
Unique category_id count: 928
Unique category_code count: 511
Unique brand count: 23022


# Counting Missing value and deal with it

In [31]:

transactions = transactions.distinct()

In [32]:
missing_counts = transactions.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in transactions.columns])


In [33]:
transactions.show(10,truncate=False)

+-------------------+-------------------+-------------------+-------------------+--------------------------------------+--------+------+-------------------+
|event_time         |order_id           |product_id         |category_id        |category_code                         |brand   |price |user_id            |
+-------------------+-------------------+-------------------+-------------------+--------------------------------------+--------+------+-------------------+
|2020-04-29 13:03:18|2298020380796780966|1515966223509089955|2268105441009468104|appliances.kitchen.meat_grinder       |moulinex|57.85 |1515915625444057633|
|2020-04-29 17:25:44|2298152465654088068|1515966223509090099|2360741866917331945|appliances.environment.air_conditioner|beko    |196.74|1515915625452306321|
|2020-04-30 03:52:02|2298467695349203486|1515966223509127913|2268105393848713950|appliances.kitchen.refrigerators      |birjusa |252.29|1515915625452503257|
|2020-04-30 07:34:21|2298579586444690133|15159662235091041

In [34]:
new_num_rows = transactions.count()
print(f"Number of rows in transactions: {new_num_rows}")

Number of rows in transactions: 2632846


In [35]:
missing_counts.show()

+----------+--------+----------+-----------+-------------+------+------+-------+
|event_time|order_id|product_id|category_id|category_code| brand| price|user_id|
+----------+--------+----------+-----------+-------------+------+------+-------+
|         0|       0|         0|     431953|       612053|505965|431953|2069351|
+----------+--------+----------+-----------+-------------+------+------+-------+



# Filling the category_code.
1. From my perspective, category_code should be mathced with category_id, means 1v1. Therefore, I decide to find the pairs of category_id and category_code to fill category_code;
2. If there is no category_code be mathched, will be given value 'na';
3. Then remove the category_code = 'na'


In [64]:

category_mapping = (
    transactions
    .select("category_id", "category_code")
    .filter(F.col("category_code").isNotNull())
    .dropDuplicates()
)


transactions_filled_code = (
    transactions
    .join(category_mapping.withColumnRenamed("category_code", "category_code_mapping"), on="category_id", how="left")
    .withColumn(
        "category_code",
        F.coalesce(F.col("category_code_mapping"), F.lit("na"))
    )
    .drop("category_code_mapping")
)


In [69]:
missing_counts2 = transactions_filled_code.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in transactions_filled_code.columns])

In [70]:
missing_counts2.show()

+-----------+----------+--------+----------+-------------+------+------+-------+
|category_id|event_time|order_id|product_id|category_code| brand| price|user_id|
+-----------+----------+--------+----------+-------------+------+------+-------+
|     431953|         0|       0|         0|            0|505965|431953|2069351|
+-----------+----------+--------+----------+-------------+------+------+-------+



In [71]:
transactions_cleaned = transactions_filled_code.filter(F.col("category_code") != "na")


In [72]:

transactions_cleaned.show(10)

+-------------------+-------------------+-------------------+-------------------+--------------------+----------------+-------+-------------------+
|        category_id|         event_time|           order_id|         product_id|       category_code|           brand|  price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+----------------+-------+-------------------+
|2374498913999872433|2020-05-27 10:40:41|2353235075740467568|1515966223523303381|  computers.notebook|       microsoft|  92.57|1515915625484634627|
|2374498913999872433|2020-03-18 05:08:49|2348821332633845779|1515966223523303381|  computers.notebook|       microsoft|  92.57|1515915625484672316|
|2374498913999872433|2020-03-13 09:02:04|2348818144560153139|1515966223523303381|  computers.notebook|       microsoft|  92.57|1515915625484669258|
|2374498913999872433|2020-02-14 09:00:43|2348796044923372273|1515966223523303381|  computers.notebook|       mic

In [76]:
num_rows2 = transactions_cleaned.count()
print(f"Number of rows in transactions: {num_rows2}")

Number of rows in transactions: 1588840


In [77]:
missing_counts3 = transactions_cleaned.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in transactions_cleaned.columns])

In [78]:
missing_counts3.show()

+-----------+----------+--------+----------+-------------+-----+-----+-------+
|category_id|event_time|order_id|product_id|category_code|brand|price|user_id|
+-----------+----------+--------+----------+-------------+-----+-----+-------+
|          0|         0|       0|         0|            0|57165|    0|1154566|
+-----------+----------+--------+----------+-------------+-----+-----+-------+



# Dealing with user_id, here I just remove that user_id is null

In [79]:
transactions_cleaned2 = transactions_cleaned.filter(F.col("user_id").isNotNull())

In [80]:

missing_counts4 = transactions_cleaned2.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in transactions_cleaned2.columns])

In [81]:
missing_counts4.show()

+-----------+----------+--------+----------+-------------+-----+-----+-------+
|category_id|event_time|order_id|product_id|category_code|brand|price|user_id|
+-----------+----------+--------+----------+-------------+-----+-----+-------+
|          0|         0|       0|         0|            0|14056|    0|      0|
+-----------+----------+--------+----------+-------------+-----+-----+-------+



# Dealing with brand, here I just remove that brand is null

In [85]:
transactions_cleaned3 = transactions_cleaned2.filter(F.col("brand").isNotNull())

In [86]:
msing_counts5 = transactions_cleaned3.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in transactions_cleaned3.columns])

In [87]:
msing_counts5.show()

+-----------+----------+--------+----------+-------------+-----+-----+-------+
|category_id|event_time|order_id|product_id|category_code|brand|price|user_id|
+-----------+----------+--------+----------+-------------+-----+-----+-------+
|          0|         0|       0|         0|            0|    0|    0|      0|
+-----------+----------+--------+----------+-------------+-----+-----+-------+



In [88]:
num_rows3 = transactions_cleaned3.count()
print(f"Number of rows in transactions: {num_rows3}")

Number of rows in transactions: 420218
